In [1]:
import tdmclient.notebook
await tdmclient.notebook.start()
from tdmclient import ClientAsync, aw
client = ClientAsync()
client.process_waiting_messages()
node = await client.wait_for_node()
aw(node.run())
aw(node.stop())
aw(node.set_scratchpad("Hello, Studio!"))
aw(node.unlock())

import time
from aled import  *
from l42 import *
from local_nav import *


In [2]:
#getters and setters functions

@tdmclient.notebook.sync_var
def get_prox_value():
    return prox_horizontal

@tdmclient.notebook.sync_var
def get_ground_value():
    return prox_ground_reflected

@tdmclient.notebook.sync_var
def get_motors_speed():
    global motor_left_target, motor_right_target
    motor_speeds = [motor_left_target, motor_right_target]
    return motor_speeds


@tdmclient.notebook.sync_var
def set_motor_speed(right_speed, left_speed):
    global motor_left_target, motor_right_target
    motor_left_target = left_speed
    motor_right_target = right_speed

In [ ]:
initial_corners = init_corner()

while(True): 
    #copy the states of the robot   
    prox_horiz_values = get_prox_value()
    prox_ground_values = get_ground_value()
    set_motor_speed(100, 100)
    motor_speeds = get_motors_speed()
    
    #Ajout Local navigation (Brendan)
    #Local obstacle avoidance avec une boucle interne de contournement
    obstacle = False
    obstacle = checkObstacle(prox_horiz_values)
    
    while(obstacle) :
        motor_speeds = avoidObstacle(prox_horiz_values, motor_speeds)
        set_motor_speed(motor_speeds[0], motor_speeds[1])
        prox_horiz_values = get_prox_value()
        obstacle = checkObstacle(prox_horiz_values)
        await client.sleep(0.2)
    #Fin de la Local navigation (Brendan)
    
    
    #open camera and update of the KF + color detection    
    if(not open_cam()):
        break
    await client.sleep(0.2)
  




test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[0, 0]
test
[-100, -100]
test
[-200, -200]
test
[-280, -280]
test
[-340, -340]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[-40, 240]
test
[-180, 380]
test
[-340, 540]
test
[100, 100]
test
[100, -20]
test
[100, -140]
test
[160, -200]
test
[320, -360]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[20, 180]
test
[-120, 200]
test
[-320, 400]
test
[-480, 560]
test
[-640, 720]
test
[-880, 960]
test
[-1080, 1160]
test
[-1320, 1400]
test
[-1520, 1600]
test
[-1680, 1760]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]
test
[100, 100]


In [ ]:
await tdmclient.notebook.stop()